# 2025美赛C题：数据预处理流程

## 1. 项目目标与数据概览

### 📌 本Notebook的目标

本文档是2025年美国大学生数学建模竞赛（MCM）C题《奥运奖牌榜模型》的**数据预处理流程**。我们的核心目标是：

1. **理解数据结构**：熟悉奥运会相关的五张核心数据表
2. **清洗与整合**：处理缺失值、异常值，并将多表合并为统一的分析主表
3. **特征工程**：创建对奖牌预测有价值的衍生特征（如历史表现、东道主效应等）
4. **为建模打基础**：输出干净、结构化的数据集，供后续预测模型使用

### 📊 数据来源

我们共有**5个数据文件**：

| 文件名 | 描述 | 核心字段 |
|--------|------|----------|
| `summerOly_medal_counts.csv` | 各国历届奖牌数（事实表） | NOC, Gold, Silver, Bronze, Total, Year |
| `summerOly_hosts.csv` | 历届奥运会主办城市 | Year, Host |
| `summerOly_programs.csv` | 各届比赛项目数量 | Sport, Discipline, 各年份列 |
| `summerOly_athletes.csv` | 运动员级别详细记录 | Name, NOC, Year, Sport, Event, Medal |
| `data_dictionary.csv` | 数据字典 | 字段说明 |

### 💡 核心理念

> **"本文档旨在通过清晰的数据处理，为奖牌预测模型打下基础。每一步处理都有其业务逻辑——我们不仅要知道'怎么做'，更要理解'为什么这样做'。"**

---

现在，让我们开始加载数据。


In [1]:
# 导入必要的库
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 设置显示选项，便于查看数据
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 200)

print("✅ 库导入成功！")


✅ 库导入成功！


In [2]:
# 读取所有数据文件
# 使用相对路径，假设Notebook与数据文件在同一目录

# 1. 奖牌数据 - 这是我们的核心事实表
df_medals = pd.read_csv('summerOly_medal_counts.csv')

# 2. 主办城市数据
df_hosts = pd.read_csv('summerOly_hosts.csv')

# 3. 比赛项目数据
# 注意：该文件包含特殊字符，需要使用latin-1编码
df_programs = pd.read_csv('summerOly_programs.csv', encoding='latin-1')

# 4. 运动员数据（文件较大，先只读取前10万行用于探索）
# 注意：完整分析时应读取全部数据
df_athletes = pd.read_csv('summerOly_athletes.csv', nrows=100000)

# ⚠️ 重要：清理数据中的特殊空格字符 \xa0 (non-breaking space)
# 原始数据中存在这种特殊字符，会导致字符串匹配失败
df_medals['NOC'] = df_medals['NOC'].str.replace('\xa0', '', regex=False).str.strip()

print("✅ 数据加载完成！")
print(f"\n各表行数统计：")
print(f"  - 奖牌数据 (df_medals): {len(df_medals):,} 行")
print(f"  - 主办城市 (df_hosts): {len(df_hosts):,} 行")
print(f"  - 比赛项目 (df_programs): {len(df_programs):,} 行")
print(f"  - 运动员数据 (df_athletes): {len(df_athletes):,} 行 (仅加载部分)")


✅ 数据加载完成！

各表行数统计：
  - 奖牌数据 (df_medals): 1,435 行
  - 主办城市 (df_hosts): 35 行
  - 比赛项目 (df_programs): 74 行
  - 运动员数据 (df_athletes): 100,000 行 (仅加载部分)


### 1.1 查看奖牌数据（核心事实表）

奖牌数据是我们分析的**核心事实表**，它记录了每届奥运会每个国家获得的金、银、铜牌数量。让我们先了解它的结构：


In [3]:
# 查看奖牌数据的基本信息
print("=" * 50)
print("📊 奖牌数据 (df_medals) 结构信息")
print("=" * 50)
df_medals.info()

print("\n" + "=" * 50)
print("📋 前10行数据预览")
print("=" * 50)
df_medals.head(10)


📊 奖牌数据 (df_medals) 结构信息
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1435 entries, 0 to 1434
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rank    1435 non-null   int64 
 1   NOC     1435 non-null   object
 2   Gold    1435 non-null   int64 
 3   Silver  1435 non-null   int64 
 4   Bronze  1435 non-null   int64 
 5   Total   1435 non-null   int64 
 6   Year    1435 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 78.6+ KB

📋 前10行数据预览


,Rank,NOC,Gold,Silver,Bronze,Total,Year
0,1,United States,11,7,2,20,1896
1,2,Greece,10,18,19,47,1896
2,3,Germany,6,5,2,13,1896
3,4,France,5,4,2,11,1896
4,5,Great Britain,2,3,2,7,1896
5,6,Hungary,2,1,3,6,1896
6,7,Austria,2,1,2,5,1896
7,8,Australia,2,0,0,2,1896
8,9,Denmark,1,2,3,6,1896
9,10,Switzerland,1,2,0,3,1896


In [4]:
# 查看数据的时间跨度和国家数量
print("📅 年份范围:", df_medals['Year'].min(), "-", df_medals['Year'].max())
print("🌍 参赛国家/地区数:", df_medals['NOC'].nunique())
print("🏅 奥运届数:", df_medals['Year'].nunique())

# 查看有哪些年份的奥运会
print("\n📆 所有奥运年份:")
print(sorted(df_medals['Year'].unique()))


📅 年份范围: 1896 - 2024
🌍 参赛国家/地区数: 164
🏅 奥运届数: 30

📆 所有奥运年份:
[np.int64(1896), np.int64(1900), np.int64(1904), np.int64(1908), np.int64(1912), np.int64(1920), np.int64(1924), np.int64(1928), np.int64(1932), np.int64(1936), np.int64(1948), np.int64(1952), np.int64(1956), np.int64(1960), np.int64(1964), np.int64(1968), np.int64(1972), np.int64(1976), np.int64(1980), np.int64(1984), np.int64(1988), np.int64(1992), np.int64(1996), np.int64(2000), np.int64(2004), np.int64(2008), np.int64(2012), np.int64(2016), np.int64(2020), np.int64(2024)]


### 1.2 查看主办城市数据

主办城市数据对于分析**"东道主效应"**至关重要。研究表明，东道国通常会获得比平常更多的奖牌。


In [5]:
# 查看主办城市数据
print("=" * 50)
print("🏠 主办城市数据 (df_hosts)")
print("=" * 50)
df_hosts.info()
print("\n")
df_hosts


🏠 主办城市数据 (df_hosts)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Year    35 non-null     int64 
 1   Host    35 non-null     object
dtypes: int64(1), object(1)
memory usage: 688.0+ bytes




,Year,Host
0,1896,"Athens, Greece"
1,1900,"Paris, France"
2,1904,"St. Louis, United States"
3,1908,"London, United Kingdom"
4,1912,"Stockholm, Sweden"
5,1916,Cancelled (WWI – Berlin had been awarded)
6,1920,"Antwerp, Belgium"
7,1924,"Paris, France"
8,1928,"Amsterdam, Netherlands"
9,1932,"Los Angeles, United States"


### 1.3 查看比赛项目数据

比赛项目数量随年份变化，这会直接影响可获得的奖牌总数。了解这一点对于**归一化奖牌数**很重要。


In [6]:
# 查看比赛项目数据结构
print("=" * 50)
print("🎯 比赛项目数据 (df_programs) 结构")
print("=" * 50)
print(f"行数: {len(df_programs)}, 列数: {len(df_programs.columns)}")
print(f"\n列名: {list(df_programs.columns[:10])}... (前10列)")

# 查看前几行
df_programs.head()


🎯 比赛项目数据 (df_programs) 结构
行数: 74, 列数: 35

列名: ['Sport', 'Discipline', 'Code', 'Sports Governing Body', '1896', '1900', '1904', '1906*', '1908', '1912']... (前10列)


,Sport,Discipline,Code,Sports Governing Body,1896,1900,1904,1906*,1908,1912,...,1988,1992,1996,2000,2004,2008,2012,2016,2020,2024
0,Aquatics,Artistic Swimming,SWA,World Aquatics,0,0,0,0,0,0,...,2,2,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,Aquatics,Diving,DIV,World Aquatics,0,0,2,1,2,4,...,4,4,4.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
2,Aquatics,Marathon Swimming,OWS,World Aquatics,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0
3,Aquatics,Swimming,SWM,World Aquatics,4,7,9,4,6,9,...,31,31,32.0,32.0,32.0,32.0,32.0,32.0,35.0,35.0
4,Aquatics,Water Polo,WPO,World Aquatics,0,1,1,0,1,1,...,1,1,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [7]:
# 查看最后几行，这里通常有汇总信息
df_programs.tail(5)


,Sport,Discipline,Code,Sports Governing Body,1896,1900,1904,1906*,1908,1912,...,1988,1992,1996,2000,2004,2008,2012,2016,2020,2024
69,Skating,Figure,FSK,ISU,0,0,0,0,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,Ice Hockey,Ice Hockey,IHO,IIHF,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,Total events,Total events,Total events,Total events,43,97,95,76,110,102,...,237,257,271.0,300.0,301.0,302.0,302.0,306.0,339.0,329.0
72,Total disciplines,Total disciplines,Total disciplines,Total disciplines,10,22,18,13,25,18,...,31,34,37.0,40.0,40.0,42.0,40.0,42.0,50.0,48.0
73,Total sports,Total sports,Total sports,Total sports,11,20,16,11,22,14,...,23,25,25.0,27.0,27.0,27.0,26.0,28.0,33.0,32.0


### 1.4 查看运动员数据

运动员数据是最详细的数据表，包含每位运动员的参赛记录。由于数据量大，我们可以用它来**验证**我们的处理结果，或进行更细粒度的分析。


In [8]:
# 查看运动员数据
print("=" * 50)
print("🏃 运动员数据 (df_athletes) 结构")
print("=" * 50)
df_athletes.info()

print("\n" + "=" * 50)
print("📋 前10行数据预览")
print("=" * 50)
df_athletes.head(10)


🏃 运动员数据 (df_athletes) 结构
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Name    100000 non-null  object
 1   Sex     100000 non-null  object
 2   Team    100000 non-null  object
 3   NOC     100000 non-null  object
 4   Year    100000 non-null  int64 
 5   City    100000 non-null  object
 6   Sport   100000 non-null  object
 7   Event   100000 non-null  object
 8   Medal   100000 non-null  object
dtypes: int64(1), object(8)
memory usage: 6.9+ MB

📋 前10行数据预览


,Name,Sex,Team,NOC,Year,City,Sport,Event,Medal
0,A Dijiang,M,China,CHN,1992,Barcelona,Basketball,Basketball Men's Basketball,No medal
1,A Lamusi,M,China,CHN,2012,London,Judo,Judo Men's Extra-Lightweight,No medal
2,Gunnar Aaby,M,Denmark,DEN,1920,Antwerpen,Football,Football Men's Football,No medal
3,Edgar Aabye,M,Denmark/Sweden,DEN,1900,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,Cornelia (-strannood),F,Netherlands,NED,1932,Los Angeles,Athletics,Athletics Women's 100 metres,No medal
5,Cornelia (-strannood),F,Netherlands,NED,1932,Los Angeles,Athletics,Athletics Women's 4 x 100 metres Relay,No medal
6,Einar Aalto,M,Finland,FIN,1952,Helsinki,Swimming,Swimming Men's 400 metres Freestyle,No medal
7,Jyri Aalto,M,Finland,FIN,2000,Sydney,Badminton,Badminton Men's Singles,No medal
8,Minna Aalto,F,Finland,FIN,1996,Atlanta,Sailing,Sailing Women's Windsurfer,No medal
9,Minna Aalto,F,Finland,FIN,2000,Sydney,Sailing,Sailing Women's Windsurfer,No medal


### 📝 1.5 数据初探总结

通过初步查看，我们观察到以下重要信息：

**关于奖牌数据 (`df_medals`)：**
- 数据覆盖1896年至2024年的夏季奥运会
- `NOC`字段使用的是国家**全名**（如"United States"），而非3字母代码
- 存在历史实体（如"Soviet Union"、"Mixed team"等）

**关于主办城市数据 (`df_hosts`)：**
- 包含了被取消的奥运会（1916、1940、1944年）
- 也包含未来的奥运会（2028、2032年）
- Host字段格式为"城市, 国家"，需要解析

**关于比赛项目数据 (`df_programs`)：**
- 这是一个**宽表**，每年的项目数作为单独的列
- 最后几行包含汇总统计（总项目数、总学科数等）
- 需要转换为**长表**格式才能与其他数据合并

**关于运动员数据 (`df_athletes`)：**
- 使用国家**3字母代码**（如"CHN"、"USA"）
- 与奖牌数据的命名方式不同，需要注意映射关系
- 可用于验证处理结果的准确性

---

接下来，我们将进入**数据清洗与合并**阶段。


---

## 2. 数据清洗与合并

### 2.1 奖牌数据清洗：处理缺失值与历史实体

#### 为什么要处理缺失值？

在奥运奖牌数据中，缺失值通常有两种业务含义：
1. **该国当届未参赛** - 应该排除在分析之外
2. **该国参赛但未获奖** - 应该填充为0

在我们的数据集中，只有**获得奖牌的国家**才会出现在记录中，因此当前数据集不存在真正的"缺失值"。但在合并其他数据后，可能会产生缺失，此时填充为0是合理的业务逻辑。

#### 关于历史实体（如苏联 URS）

历史上存在一些已解体或改名的国家/地区：
- **苏联 (Soviet Union)** → 解体后分为俄罗斯、乌克兰等多国
- **东德/西德** → 统一为德国
- **南斯拉夫** → 分裂为塞尔维亚、克罗地亚等

**我们的处理策略**：保留原始记录作为独立特征，而非简单合并。原因是：
1. 历史表现是重要的预测信号
2. 简单合并会扭曲历史数据的统计特征
3. 模型可以学习到"继承国"与"前身国"之间的关系


In [9]:
# 检查奖牌数据的缺失值情况
print("=" * 50)
print("🔍 奖牌数据缺失值检查")
print("=" * 50)
print(df_medals.isnull().sum())

print("\n" + "=" * 50)
print("📊 奖牌数据基本统计")
print("=" * 50)
df_medals.describe()


🔍 奖牌数据缺失值检查
Rank      0
NOC       0
Gold      0
Silver    0
Bronze    0
Total     0
Year      0
dtype: int64

📊 奖牌数据基本统计


,Rank,Gold,Silver,Bronze,Total,Year
count,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000
mean,30.334495,4.048084,4.025784,4.384669,12.458537,1981.828571
std,21.920604,8.390142,7.100987,6.822294,21.582579,34.188111
min,1.000000,0.000000,0.000000,0.000000,1.000000,1896.000000
25%,12.000000,0.000000,0.000000,1.000000,2.000000,1960.000000
50%,26.000000,1.000000,2.000000,2.000000,5.000000,1992.000000
75%,43.500000,4.000000,4.000000,5.000000,13.000000,2008.000000
max,86.000000,83.000000,78.000000,77.000000,231.000000,2024.000000


In [10]:
# 检查重复值
duplicates = df_medals.duplicated(subset=['NOC', 'Year'], keep=False)
print(f"🔄 是否存在同一国家同一年的重复记录: {duplicates.any()}")

if duplicates.any():
    print("\n重复记录示例:")
    print(df_medals[duplicates].head(10))


🔄 是否存在同一国家同一年的重复记录: False


In [11]:
# 查看有哪些特殊的国家/地区实体
print("=" * 50)
print("🌍 所有国家/地区实体列表 (按名称排序)")
print("=" * 50)
all_countries = sorted(df_medals['NOC'].unique())
print(f"共 {len(all_countries)} 个实体：\n")
for i, c in enumerate(all_countries, 1):
    print(f"{i:3d}. {c}")


🌍 所有国家/地区实体列表 (按名称排序)
共 164 个实体：

  1. Afghanistan
  2. Albania
  3. Algeria
  4. Argentina
  5. Armenia
  6. Australasia
  7. Australia
  8. Austria
  9. Azerbaijan
 10. Bahamas
 11. Bahrain
 12. Barbados
 13. Belarus
 14. Belgium
 15. Bermuda
 16. Bohemia
 17. Botswana
 18. Brazil
 19. British West Indies
 20. Bulgaria
 21. Burkina Faso
 22. Burundi
 23. Cabo Verde
 24. Cameroon
 25. Canada
 26. Ceylon
 27. Chile
 28. China
 29. Chinese Taipei
 30. Colombia
 31. Costa Rica
 32. Croatia
 33. Cuba
 34. Cyprus
 35. Czech Republic
 36. Czechoslovakia
 37. Denmark
 38. Djibouti
 39. Dominica
 40. Dominican Republic
 41. East Germany
 42. Ecuador
 43. Egypt
 44. Eritrea
 45. Estonia
 46. Ethiopia
 47. FR Yugoslavia
 48. Fiji
 49. Finland
 50. Formosa
 51. France
 52. Gabon
 53. Georgia
 54. Germany
 55. Ghana
 56. Great Britain
 57. Greece
 58. Grenada
 59. Guatemala
 60. Guyana
 61. Haiti
 62. Hong Kong
 63. Hungary
 64. Iceland
 65. Independent Olympic Athletes
 66. Independent Olympic P

In [12]:
# 识别可能的历史实体（通过名称特征识别）
historical_keywords = ['Soviet', 'Yugoslavia', 'Czechoslov', 'East Germany', 'West Germany', 
                       'United Team', 'Mixed team', 'Independent', 'Refugee']

print("=" * 50)
print("📜 疑似历史实体或特殊实体")
print("=" * 50)

for country in all_countries:
    for keyword in historical_keywords:
        if keyword.lower() in country.lower():
            # 查看该实体的参赛年份范围
            years = df_medals[df_medals['NOC'] == country]['Year']
            print(f"  - {country}: {years.min()}-{years.max()} ({len(years)}届)")
            break


📜 疑似历史实体或特殊实体
  - Czechoslovakia: 1920-1992 (16届)
  - East Germany: 1968-1988 (5届)
  - FR Yugoslavia: 1996-2000 (2届)
  - Independent Olympic Athletes: 2016-2016 (1届)
  - Independent Olympic Participants: 1992-1992 (1届)
  - Mixed team: 1896-1904 (3届)
  - Refugee Olympic Team: 2024-2024 (1届)
  - Soviet Union: 1952-1988 (9届)
  - United Team of Germany: 1956-1964 (3届)
  - West Germany: 1968-1988 (5届)
  - Yugoslavia: 1924-1988 (14届)


### 2.2 核心数据合并：构建分析主表

#### 合并策略

我们将以 `df_medals`（奖牌数据）为**事实表**，依次合并其他维度表：

1. **左连接 `hosts` 表** → 添加东道主标志
2. **左连接 `programs` 汇总数据** → 添加当届总项目数

#### 为什么选择左连接（LEFT JOIN）？

- **保留所有国家的奖牌记录**：即使某些年份的主办信息缺失，也不应丢失奖牌数据
- **避免数据丢失**：内连接（INNER JOIN）可能因键不匹配而丢失数据

#### 关于"东道主效应"

研究表明，东道国通常会获得比平常多20-30%的奖牌。可能原因包括：
- 主场观众支持
- 无需长途旅行，运动员状态更好
- 东道国通常会增加体育投入
- 部分项目可能存在裁判倾向


In [13]:
# 首先，清理hosts数据并提取东道国信息
print("=" * 50)
print("🏠 处理主办城市数据")
print("=" * 50)

# 查看Host字段的格式（注意：原始数据中使用了特殊空格字符\xa0）
print("原始Host字段示例:")
print(df_hosts['Host'].head(10).tolist())

# 创建hosts的副本进行处理
df_hosts_clean = df_hosts.copy()

# 过滤掉被取消的奥运会（Cancelled）
df_hosts_clean = df_hosts_clean[~df_hosts_clean['Host'].str.contains('Cancelled', na=False)]

# 从Host字段提取国家名称
# 注意：原始数据使用了\xa0（非断行空格），需要特殊处理
def extract_country(host_str):
    """从Host字段提取国家名称"""
    if pd.isna(host_str):
        return None
    # 先替换特殊空格字符为普通空格，再去除前后空格
    host_str = host_str.replace('\xa0', ' ').strip()
    # 用逗号分割，取最后一部分（国家名）
    parts = host_str.split(',')
    if len(parts) >= 2:
        country = parts[-1].strip()
        # 处理特殊情况如 "Japan (postponed...)"
        if '(' in country:
            country = country.split('(')[0].strip()
        return country
    return host_str

df_hosts_clean['Host_Country'] = df_hosts_clean['Host'].apply(extract_country)

print("\n提取的东道国:")
print(df_hosts_clean[['Year', 'Host', 'Host_Country']].head(15))


🏠 处理主办城市数据
原始Host字段示例:
['\xa0Athens,\xa0Greece', '\xa0Paris,\xa0France', '\xa0St. Louis,\xa0United States', '\xa0London,\xa0United Kingdom', '\xa0Stockholm,\xa0Sweden', '\xa0Cancelled (WWI – Berlin had been awarded)', '\xa0Antwerp,\xa0Belgium', '\xa0Paris,\xa0France', '\xa0Amsterdam,\xa0Netherlands', '\xa0Los Angeles,\xa0United States']

提取的东道国:
    Year                         Host    Host_Country
0   1896               Athens, Greece          Greece
1   1900                Paris, France          France
2   1904     St. Louis, United States   United States
3   1908       London, United Kingdom  United Kingdom
4   1912            Stockholm, Sweden          Sweden
6   1920             Antwerp, Belgium         Belgium
7   1924                Paris, France          France
8   1928       Amsterdam, Netherlands     Netherlands
9   1932   Los Angeles, United States   United States
10  1936              Berlin, Germany         Germany
13  1948       London, United Kingdom  United Kingdom
14  

In [16]:
# 创建国家名称映射表（用于匹配不同命名方式）
# 注意：df_medals中使用全名，df_hosts中也使用全名，但可能存在细微差异

# 查看hosts中的国家名称
print("hosts中的国家名称:")
print(sorted(df_hosts_clean['Host_Country'].unique()))

# 创建名称标准化映射
country_name_mapping = {
    'United States': 'United States',
    'United Kingdom': 'Great Britain',  # 奥运会中英国使用"Great Britain"
    'Soviet Union': 'Soviet Union',
    'West Germany': 'West Germany',
    'South Korea': 'South Korea',
    'Greece': 'Greece',
    'France': 'France',
    'Sweden': 'Sweden',
    'Belgium': 'Belgium',
    'Netherlands': 'Netherlands',
    'Germany': 'Germany',
    'Finland': 'Finland',
    'Australia': 'Australia',
    'Italy': 'Italy',
    'Japan': 'Japan',
    'Mexico': 'Mexico',
    'Canada': 'Canada',
    'Spain': 'Spain',
    'China': 'China',
    'Brazil': 'Brazil',
}

# 应用映射
df_hosts_clean['Host_Country_Mapped'] = df_hosts_clean['Host_Country'].map(
    lambda x: country_name_mapping.get(x, x)
)

print("\n映射后的国家名称:")
print(df_hosts_clean[['Year', 'Host_Country', 'Host_Country_Mapped']].head(15))


hosts中的国家名称:
['Australia', 'Belgium', 'Brazil', 'Canada', 'China', 'Finland', 'France', 'Germany', 'Greece', 'Italy', 'Japan', 'Mexico', 'Netherlands', 'South Korea', 'Soviet Union', 'Spain', 'Sweden', 'United Kingdom', 'United States', 'West Germany']

映射后的国家名称:
    Year    Host_Country Host_Country_Mapped
0   1896          Greece              Greece
1   1900          France              France
2   1904   United States       United States
3   1908  United Kingdom       Great Britain
4   1912          Sweden              Sweden
6   1920         Belgium             Belgium
7   1924          France              France
8   1928     Netherlands         Netherlands
9   1932   United States       United States
10  1936         Germany             Germany
13  1948  United Kingdom       Great Britain
14  1952         Finland             Finland
15  1956       Australia           Australia
16  1960           Italy               Italy
17  1964           Japan               Japan


In [17]:
# 开始构建主表
# 第一步：复制奖牌数据作为基础
df_main = df_medals.copy()

# 第二步：添加东道主标志
# 创建year-host映射
year_host_map = dict(zip(df_hosts_clean['Year'], df_hosts_clean['Host_Country_Mapped']))

# 判断是否为东道主：当年的东道国与该国名称匹配
df_main['is_host'] = df_main.apply(
    lambda row: 1 if year_host_map.get(row['Year']) == row['NOC'] else 0,
    axis=1
)

print("=" * 50)
print("🏠 东道主标志添加完成")
print("=" * 50)
print(f"\n东道主记录数: {df_main['is_host'].sum()}")
print(f"非东道主记录数: {(df_main['is_host'] == 0).sum()}")

# 查看东道主记录
print("\n东道主记录示例:")
print(df_main[df_main['is_host'] == 1][['Year', 'NOC', 'Gold', 'Silver', 'Bronze', 'Total', 'is_host']])


🏠 东道主标志添加完成

东道主记录数: 30
非东道主记录数: 1405

东道主记录示例:
      Year            NOC  Gold  Silver  Bronze  Total  is_host
1     1896         Greece    10      18      19     47        1
11    1900         France    27      39      37    103        1
32    1904  United States    76      78      77    231        1
45    1908  Great Britain    56      51      39    146        1
65    1912         Sweden    23      25      17     65        1
86    1920        Belgium    14      11      11     36        1
106   1924         France    13      15      10     38        1
138   1928    Netherlands     6       9       4     19        1
164   1932  United States    44      36      30    110        1
192   1936        Germany    38      31      32    101        1
235   1948  Great Britain     3      14       6     23        1
268   1952        Finland     6       3      13     22        1
306   1956      Australia    13       8      14     35        1
344   1960          Italy    13      10      13     36  

In [18]:
# 第三步：添加当届总项目数
# 从programs数据中提取总项目数（最后几行有汇总）

# 找到"Total events"行
total_events_row = df_programs[df_programs['Sport'] == 'Total events']
print("总项目数行:")
print(total_events_row)

# 提取各年份的总项目数（列名是年份）
# 获取年份列（从第5列开始，跳过Sport, Discipline, Code, Sports Governing Body）
year_columns = [col for col in df_programs.columns[4:] if col.replace('*', '').isdigit()]
print(f"\n年份列: {year_columns}")


总项目数行:
           Sport    Discipline          Code Sports Governing Body 1896 1900 1904  1906*  1908 1912  ...  1988 1992   1996   2000   2004   2008   2012   2016   2020   2024
71  Total events  Total events  Total events          Total events   43   97   95     76   110  102  ...   237  257  271.0  300.0  301.0  302.0  302.0  306.0  339.0  329.0

[1 rows x 35 columns]

年份列: ['1896', '1900', '1904', '1906*', '1908', '1912', '1920', '1924', '1928', '1932', '1936', '1948', '1952', '1956', '1960', '1964', '1968', '1972', '1976', '1980', '1984', '1988', '1992', '1996', '2000', '2004', '2008', '2012', '2016', '2020', '2024']


In [19]:
# 创建年份到总项目数的映射
total_events_row = df_programs[df_programs['Sport'] == 'Total events'].iloc[0]

year_to_events = {}
for col in year_columns:
    # 处理年份列名（可能有*号，如1906*）
    year = int(col.replace('*', ''))
    try:
        events = int(total_events_row[col])
        year_to_events[year] = events
    except (ValueError, TypeError):
        pass

print("各届奥运会总项目数:")
for year, events in sorted(year_to_events.items()):
    print(f"  {year}: {events}项")


各届奥运会总项目数:
  1896: 43项
  1900: 97项
  1904: 95项
  1906: 76项
  1908: 110项
  1912: 102项
  1920: 156项
  1924: 126项
  1928: 109项
  1932: 117项
  1936: 129项
  1948: 136项
  1952: 149项
  1956: 151项
  1960: 150项
  1964: 163项
  1968: 172项
  1972: 195项
  1976: 198项
  1980: 203项
  1984: 221项
  1988: 237项
  1992: 257项
  1996: 271项
  2000: 300项
  2004: 301项
  2008: 302项
  2012: 302项
  2016: 306项
  2020: 339项
  2024: 329项


In [20]:
# 将总项目数添加到主表
df_main['total_events'] = df_main['Year'].map(year_to_events)

print("=" * 50)
print("🎯 总项目数添加完成")
print("=" * 50)

# 检查是否有缺失
print(f"\n缺失值检查:")
print(df_main['total_events'].isnull().sum(), "条记录缺少总项目数")

# 查看缺失的年份
if df_main['total_events'].isnull().any():
    print("\n缺失总项目数的年份:")
    print(df_main[df_main['total_events'].isnull()]['Year'].unique())


🎯 总项目数添加完成

缺失值检查:
0 条记录缺少总项目数


In [ ]:
# 计算奖牌占比（归一化指标）
# 这个指标消除了不同届次项目数量差异的影响
df_main['medal_ratio'] = df_main['Total'] / df_main['total_events']

print("=" * 50)
print("📊 主表构建完成 - 查看结果")
print("=" * 50)
print(f"\n主表形状: {df_main.shape}")
print(f"列名: {list(df_main.columns)}")

# 查看样例数据
print("\n样例数据（2024年）:")
df_main[df_main['Year'] == 2024].head(10)


### 2.3 合并结果验证

数据合并后，我们需要验证结果的正确性。主要检查：

1. **东道主标志**：每届应该只有一个东道主（特殊情况除外）
2. **项目数一致性**：与官方数据对照
3. **数据完整性**：无意外的缺失值


In [21]:
# 验证1：检查每届的东道主数量
print("=" * 50)
print("✅ 验证1：每届东道主数量")
print("=" * 50)

host_counts = df_main.groupby('Year')['is_host'].sum()
print("\n每届东道主数量:")
print(host_counts[host_counts > 0])  # 只显示有东道主的年份

# 检查是否有异常（一届多个或零个东道主）
abnormal = host_counts[(host_counts > 1) | (host_counts == 0)]
if len(abnormal) > 0:
    print(f"\n⚠️ 异常年份（东道主数量不为1）: {list(abnormal.index)}")
else:
    print("\n✅ 所有年份的东道主数量正常")


✅ 验证1：每届东道主数量

每届东道主数量:
Year
1896    1
1900    1
1904    1
1908    1
1912    1
1920    1
1924    1
1928    1
1932    1
1936    1
1948    1
1952    1
1956    1
1960    1
1964    1
1968    1
1972    1
1976    1
1980    1
1984    1
1988    1
1992    1
1996    1
2000    1
2004    1
2008    1
2012    1
2016    1
2020    1
2024    1
Name: is_host, dtype: int64

✅ 所有年份的东道主数量正常


In [22]:
# 验证2：检查数据完整性
print("=" * 50)
print("✅ 验证2：数据完整性检查")
print("=" * 50)

print("\n各列缺失值统计:")
print(df_main.isnull().sum())

print("\n数据类型:")
print(df_main.dtypes)


✅ 验证2：数据完整性检查

各列缺失值统计:
Rank            0
NOC             0
Gold            0
Silver          0
Bronze          0
Total           0
Year            0
is_host         0
total_events    0
dtype: int64

数据类型:
Rank             int64
NOC             object
Gold             int64
Silver           int64
Bronze           int64
Total            int64
Year             int64
is_host          int64
total_events     int64
dtype: object


In [23]:
# 验证3：东道主效应初步观察
print("=" * 50)
print("📊 验证3：东道主效应初步观察")
print("=" * 50)

# 比较东道主与非东道主的平均奖牌数
host_comparison = df_main.groupby('is_host').agg({
    'Total': ['mean', 'median', 'std'],
    'Gold': 'mean',
    'medal_ratio': 'mean'
}).round(2)

host_comparison.columns = ['平均总奖牌', '中位数总奖牌', '标准差', '平均金牌', '平均奖牌占比']
host_comparison.index = ['非东道主', '东道主']
print(host_comparison)

print("\n💡 初步结论：东道主平均获得的奖牌数明显高于非东道主国家。")


📊 验证3：东道主效应初步观察


KeyError: "Column(s) ['medal_ratio'] do not exist"

---

## 3. 特征工程

特征工程是机器学习中最关键的环节之一。好的特征可以极大地提升模型性能。

### 3.1 创建滞后特征：捕捉历史表现

#### 什么是滞后特征？

滞后特征（Lag Features）是将**过去的值**作为当前的特征。在奖牌预测中：
- `lag1`：上一届奥运会的奖牌数
- `lag2`：上两届奥运会的奖牌数

#### 为什么滞后特征重要？

1. **惯性效应**：强国往往持续强势，弱国短期内难以逆袭
2. **政策周期**：体育投入的效果有滞后性（4-8年）
3. **预测基准**："与上届相同"本身就是一个不错的预测基线

#### 如何处理首次参赛的国家？

对于首次参赛的国家，滞后值为空（NaN）。我们选择填充为0，业务含义是"无历史成绩"。

> ⚠️ **重要**：使用 `shift(1)` 而非 `shift(-1)`！
> - `shift(1)` 向下移动数据，获取**上一行**的值（过去）
> - `shift(-1)` 向上移动数据，获取**下一行**的值（未来）
> - 在预测场景中，我们只能使用过去的信息！


In [24]:
# 首先，确保数据按国家和年份排序
df_main = df_main.sort_values(['NOC', 'Year']).reset_index(drop=True)

print("=" * 50)
print("📊 创建滞后特征")
print("=" * 50)

# 创建lag1特征（上一届奥运会的奖牌数）
# groupby确保只在同一国家内进行shift操作
df_main['total_lag1'] = df_main.groupby('NOC')['Total'].shift(1)
df_main['gold_lag1'] = df_main.groupby('NOC')['Gold'].shift(1)

# 创建lag2特征（上两届奥运会的奖牌数）
df_main['total_lag2'] = df_main.groupby('NOC')['Total'].shift(2)

print("滞后特征创建完成！")
print(f"\n缺失值统计（预期：首次/首两次参赛的记录）:")
print(f"  - total_lag1 缺失: {df_main['total_lag1'].isnull().sum()} 条")
print(f"  - total_lag2 缺失: {df_main['total_lag2'].isnull().sum()} 条")


📊 创建滞后特征
滞后特征创建完成！

缺失值统计（预期：首次/首两次参赛的记录）:
  - total_lag1 缺失: 164 条
  - total_lag2 缺失: 291 条


In [25]:
# 填充缺失值为0（表示无历史成绩）
df_main['total_lag1'] = df_main['total_lag1'].fillna(0)
df_main['gold_lag1'] = df_main['gold_lag1'].fillna(0)
df_main['total_lag2'] = df_main['total_lag2'].fillna(0)

print("✅ 缺失值已填充为0")


✅ 缺失值已填充为0


In [26]:
# 验证滞后特征的正确性（以美国为例）
print("=" * 50)
print("🔍 验证：美国(United States)的滞后特征")
print("=" * 50)

usa_data = df_main[df_main['NOC'] == 'United States'][['Year', 'Total', 'total_lag1', 'total_lag2']].tail(10)
print(usa_data)

print("\n💡 验证逻辑：每行的total_lag1应等于上一行的Total值")


🔍 验证：美国(United States)的滞后特征
      Year  Total  total_lag1  total_lag2
1365  1988     94       174.0        94.0
1366  1992    108        94.0       174.0
1367  1996    101       108.0        94.0
1368  2000     93       101.0       108.0
1369  2004    101        93.0       101.0
1370  2008    112       101.0        93.0
1371  2012    104       112.0       101.0
1372  2016    121       104.0       112.0
1373  2020    113       121.0       104.0
1374  2024    126       113.0       121.0

💡 验证逻辑：每行的total_lag1应等于上一行的Total值


### 3.2 创建滚动平均特征：衡量稳定水平

#### 为什么需要滚动平均？

单届的奖牌数可能受到偶然因素影响（如主力运动员受伤、项目设置变化等）。过去N届的**滚动平均**可以：

1. **平滑波动**：消除单届的偶然因素
2. **衡量稳定实力**：反映一个国家的持续竞争力
3. **捕捉趋势**：通过比较滚动平均与单届值，可以判断是上升还是下降趋势

#### 滚动窗口的选择

我们选择**过去3届**的滚动平均：
- 窗口太小（如2届）：平滑效果不足
- 窗口太大（如5届）：对近期变化不敏感
- 3届约12年，正好覆盖一个体育投入周期

#### 避免数据泄露

> ⚠️ **关键**：滚动平均也需要 `shift(1)`！
> 
> 如果直接使用 `rolling(3).mean()`，会包含当届的数据，造成**数据泄露**（使用了未来信息来预测未来）。
> 
> 正确做法：先 `shift(1)` 再 `rolling(3).mean()`，或使用 `rolling(3).mean().shift(1)`


In [27]:
# 创建滚动平均特征
print("=" * 50)
print("📊 创建滚动平均特征")
print("=" * 50)

# 过去3届的平均奖牌数（不包含当届）
# rolling(3).mean() 计算包含当前行在内的3行平均
# 再 shift(1) 向下移动一行，使得每行的值是"过去3届"的平均
df_main['total_rolling3_mean'] = df_main.groupby('NOC')['Total'].transform(
    lambda x: x.rolling(window=3, min_periods=1).mean().shift(1)
)

# 填充缺失值
df_main['total_rolling3_mean'] = df_main['total_rolling3_mean'].fillna(0)

print("✅ 滚动平均特征创建完成！")


📊 创建滚动平均特征
✅ 滚动平均特征创建完成！


In [28]:
# 创建额外的有用特征

# 1. 奖牌变化量（与上届相比）
df_main['total_change'] = df_main['Total'] - df_main['total_lag1']

# 2. 参赛届数（经验积累）
df_main['participation_count'] = df_main.groupby('NOC').cumcount() + 1

# 3. 金牌占比（衡量竞技水平质量）
df_main['gold_ratio'] = df_main['Gold'] / df_main['Total'].replace(0, np.nan)
df_main['gold_ratio'] = df_main['gold_ratio'].fillna(0)

print("=" * 50)
print("✅ 额外特征创建完成")
print("=" * 50)
print(f"\n当前特征列表: {list(df_main.columns)}")


✅ 额外特征创建完成

当前特征列表: ['Rank', 'NOC', 'Gold', 'Silver', 'Bronze', 'Total', 'Year', 'is_host', 'total_events', 'total_lag1', 'gold_lag1', 'total_lag2', 'total_rolling3_mean', 'total_change', 'participation_count', 'gold_ratio']


### 3.3 新特征有效性检查

特征工程完成后，我们需要验证新特征的有效性。我们选取两个代表性国家进行检查：

- **美国 (United States)**：传统体育强国，奖牌数稳定在高位
- **中国 (China)**：快速崛起的体育强国，有明显的上升趋势

通过观察这两个国家的特征值，可以直观验证特征计算的正确性。


In [29]:
# 查看美国的特征数据
print("=" * 50)
print("🇺🇸 美国 (United States) 近10届数据")
print("=" * 50)

usa_features = df_main[df_main['NOC'] == 'United States'][
    ['Year', 'Total', 'Gold', 'total_lag1', 'total_rolling3_mean', 
     'total_change', 'participation_count', 'is_host']
].tail(10)

print(usa_features.to_string(index=False))


🇺🇸 美国 (United States) 近10届数据
 Year  Total  Gold  total_lag1  total_rolling3_mean  total_change  participation_count  is_host
 1988     94    36       174.0           120.666667         -80.0                   20        0
 1992    108    37        94.0           120.666667          14.0                   21        0
 1996    101    44       108.0           125.333333          -7.0                   22        1
 2000     93    37       101.0           101.000000          -8.0                   23        0
 2004    101    36        93.0           100.666667           8.0                   24        0
 2008    112    36       101.0            98.333333          11.0                   25        0
 2012    104    48       112.0           102.000000          -8.0                   26        0
 2016    121    46       104.0           105.666667          17.0                   27        0
 2020    113    39       121.0           112.333333          -8.0                   28        0
 2024    12

In [30]:
# 查看中国的特征数据
print("=" * 50)
print("🇨🇳 中国 (China) 全部数据")
print("=" * 50)

china_features = df_main[df_main['NOC'] == 'China'][
    ['Year', 'Total', 'Gold', 'total_lag1', 'total_rolling3_mean', 
     'total_change', 'participation_count', 'is_host']
]

print(china_features.to_string(index=False))

print("\n💡 观察：")
print("  - 中国1984年首次参赛，所以lag1=0")
print("  - 2008年is_host=1，奖牌数达到峰值100枚")
print("  - 奖牌数整体呈上升趋势")


🇨🇳 中国 (China) 全部数据
 Year  Total  Gold  total_lag1  total_rolling3_mean  total_change  participation_count  is_host
 1984     32    15         0.0             0.000000          32.0                    1        0
 1988     28     5        32.0            32.000000          -4.0                    2        0
 1992     54    16        28.0            30.000000          26.0                    3        0
 1996     50    16        54.0            38.000000          -4.0                    4        0
 2000     58    28        50.0            44.000000           8.0                    5        0
 2004     63    32        58.0            54.000000           5.0                    6        0
 2008    100    48        63.0            57.000000          37.0                    7        1
 2012     92    39       100.0            73.666667          -8.0                    8        0
 2016     70    26        92.0            85.000000         -22.0                    9        0
 2020     89    38   

In [31]:
# 特征相关性检查
print("=" * 50)
print("📊 特征与目标变量(Total)的相关性")
print("=" * 50)

numeric_cols = ['Total', 'Gold', 'total_lag1', 'gold_lag1', 'total_lag2', 
                'total_rolling3_mean', 'is_host', 'total_events', 'participation_count']

correlation = df_main[numeric_cols].corr()['Total'].sort_values(ascending=False)
print(correlation)

print("\n💡 结论：")
print("  - total_lag1与Total高度相关，说明历史表现是强预测因子")
print("  - total_rolling3_mean也有很高相关性，平滑特征有效")
print("  - is_host相关性较低，但作为类别特征仍有意义")


📊 特征与目标变量(Total)的相关性
Total                  1.000000
Gold                   0.970321
total_rolling3_mean    0.834585
total_lag1             0.800085
gold_lag1              0.791012
total_lag2             0.758923
is_host                0.368497
participation_count    0.232037
total_events          -0.036603
Name: Total, dtype: float64

💡 结论：
  - total_lag1与Total高度相关，说明历史表现是强预测因子
  - total_rolling3_mean也有很高相关性，平滑特征有效
  - is_host相关性较低，但作为类别特征仍有意义


---

## 4. 数据保存与总结

### 4.1 保存处理后的数据

我们将清洗和特征工程后的数据保存为CSV文件，供后续建模使用。


In [32]:
# 保存主表
output_file = 'processed_medal_data.csv'
df_main.to_csv(output_file, index=False)

print("=" * 50)
print("💾 数据保存完成")
print("=" * 50)
print(f"\n文件名: {output_file}")
print(f"数据形状: {df_main.shape}")
print(f"\n列信息:")
for col in df_main.columns:
    print(f"  - {col}: {df_main[col].dtype}")


💾 数据保存完成

文件名: processed_medal_data.csv
数据形状: (1435, 16)

列信息:
  - Rank: int64
  - NOC: object
  - Gold: int64
  - Silver: int64
  - Bronze: int64
  - Total: int64
  - Year: int64
  - is_host: int64
  - total_events: int64
  - total_lag1: float64
  - gold_lag1: float64
  - total_lag2: float64
  - total_rolling3_mean: float64
  - total_change: float64
  - participation_count: int64
  - gold_ratio: float64


In [33]:
# 最终数据预览
print("=" * 50)
print("📋 最终数据预览（随机10条）")
print("=" * 50)
df_main.sample(10)


📋 最终数据预览（随机10条）


,Rank,NOC,Gold,Silver,Bronze,Total,Year,is_host,total_events,total_lag1,gold_lag1,total_lag2,total_rolling3_mean,total_change,participation_count,gold_ratio
1075,23,Romania,1,1,2,4,1952,0,149,1.0,0.0,1.0,1.000000,3.0,3,0.250000
749,23,Japan,3,6,5,14,1996,0,271,22.0,3.0,14.0,22.666667,-8.0,16,0.214286
522,9,Great Britain,5,7,5,17,1932,0,117,20.0,3.0,34.0,32.000000,-3.0,9,0.294118
891,8,Netherlands,6,9,4,19,1928,1,109,10.0,4.0,11.0,8.000000,9.0,6,0.315789
1071,5,ROC,20,28,23,71,2020,0,339,0.0,0.0,0.0,0.000000,71.0,1,0.281690
324,8,Czechoslovakia,6,2,3,11,1948,0,136,9.0,3.0,6.0,8.000000,2.0,6,0.545455
168,41,Brazil,0,0,2,2,1972,0,195,3.0,0.0,1.0,2.000000,-1.0,8,0.000000
517,1,Great Britain,56,51,39,146,1908,1,110,2.0,1.0,31.0,13.333333,144.0,4,0.383562
1163,19,South Korea,1,1,4,6,1976,0,198,1.0,0.0,2.0,2.000000,5.0,7,0.166667
1033,23,Poland,3,2,5,10,2004,0,301,14.0,6.0,17.0,16.666667,-4.0,18,0.300000


### 4.2 预处理总结

#### ✅ 本Notebook完成的工作

1. **数据加载与探索**
   - 加载了5个数据文件
   - 了解了各表的结构和关系
   - 发现了数据中的特殊情况（历史实体、命名差异等）

2. **数据清洗与合并**
   - 处理了主办城市数据，提取东道国信息
   - 创建了国家名称映射，解决命名不一致问题
   - 构建了包含东道主标志和总项目数的主分析表

3. **特征工程**
   - 创建了滞后特征（lag1, lag2）：捕捉历史表现
   - 创建了滚动平均特征：衡量稳定水平
   - 创建了辅助特征：参赛届数、金牌占比、奖牌变化量等

4. **数据验证**
   - 验证了东道主标志的正确性
   - 检查了特征的相关性
   - 通过具体国家案例验证了特征计算逻辑

#### 📊 最终数据集特征说明

| 特征名 | 说明 | 类型 |
|--------|------|------|
| Year | 奥运年份 | 时间 |
| NOC | 国家名称 | 类别 |
| Gold/Silver/Bronze/Total | 各类奖牌数 | 目标变量 |
| is_host | 是否为东道主 | 二元特征 |
| total_events | 当届总项目数 | 数值 |
| total_lag1/lag2 | 上1/2届奖牌数 | 滞后特征 |
| total_rolling3_mean | 过去3届平均 | 滚动特征 |
| participation_count | 参赛届数 | 经验特征 |
| medal_ratio | 奖牌占比 | 归一化特征 |
| gold_ratio | 金牌占比 | 质量特征 |

#### 🚀 下一步建议

1. **探索性数据分析（EDA）**：可视化各特征的分布和趋势
2. **特征选择**：使用统计方法或模型筛选最重要的特征
3. **模型训练**：尝试线性回归、随机森林、XGBoost等模型
4. **2028预测**：基于历史模式预测2028洛杉矶奥运会的奖牌分布


In [34]:
print("\n" + "="*60)
print("🎉 恭喜！数据预处理流程完成！")
print("="*60)
print("\n现在你已经有了一个干净、结构化的数据集，可以开始建模了！")
print("\n记住：好的数据预处理是成功建模的一半。")



🎉 恭喜！数据预处理流程完成！

现在你已经有了一个干净、结构化的数据集，可以开始建模了！

记住：好的数据预处理是成功建模的一半。
